In [1]:
import pandas as pd
import os

In [ ]:
pd.read_csv("../download/page_views.csv", nrows = 10).head()

In [ ]:
pd.read_csv("../download/events.csv", nrows = 10).head()

In [2]:
filename = "../generated/events_traffic_source.hdf5"

## Load page_view: to add traffic_source to events and check common columns
platform and traffic_source to be taken fom page_view

In [ ]:
events = pd.read_csv("../download/events.csv", dtype={'display_id':int, 'uuid':str, 'document_id':int,  'timestamp':int, 'platform':str, 'geo_location':str})

chunksize = 20 * 10 ** 6

try:
    os.remove(filename)
except OSError:
    pass

loop = 0
usecols=['uuid', 'document_id', 'timestamp', 'platform', 'traffic_source']
dtype={'uuid':str, 'document_id':int, 'timestamp':int, 'platform':str, 'traffic_source':float}

for chunk in pd.read_csv("../download/page_views.csv", chunksize=chunksize, usecols=usecols, dtype=dtype):
    #only geo_location has NaN ~150 per 10e6 rows: print len(chunk[chunk['uuid'].isnull()]), len(chunk[chunk['document_id'].isnull()]), len(chunk[chunk['timestamp'].isnull()]), len(chunk[chunk['platform'].isnull()]), len(chunk[chunk['geo_location'].isnull()]), len(chunk[chunk['traffic_source'].isnull()])
    events = events.merge(chunk, on=['uuid', 'document_id', 'timestamp'], how='left')
    index = pd.notnull(events['traffic_source'])
    events[index].drop(['uuid'], axis=1).to_hdf(filename, key='default', mode='a', append=True, complevel=5, complib='zlib')
    
    events = events[~index].rename(columns={'platform_x':'platform'}).drop(['traffic_source', 'platform_y'], axis=1)
    
    loop += 1
    
    print loop, len(events)
    if len(events)==0:
        break

events.to_hdf(filename+'.missing', key='default', mode='w', complevel=5, complib='zlib')     

print 'Done'

Process not joined timestamps

In [ ]:
events = pd.read_hdf(filename+'.missing')

loop = 0
usecols=['uuid', 'document_id', 'platform', 'traffic_source']  #no timestamp
dtype={'uuid':str, 'document_id':int, 'platform':str, 'traffic_source':float}

for chunk in pd.read_csv("../download/page_views.csv", chunksize=chunksize, usecols=usecols, dtype=dtype):
    events = events.merge(chunk, on=['uuid', 'document_id'], how='left') #no timestamp
    index = pd.notnull(events['traffic_source'])
    events[index].drop_duplicates(keep='last', subset = ['display_id']).drop(['uuid'], axis=1).to_hdf(filename, key='default', mode='a', append=True, complevel=5, complib='zlib')
    
    events = events[~index].rename(columns={'platform_x':'platform'}).drop(['traffic_source', 'platform_y'], axis=1)
    
    loop += 1
    
    print loop, len(events)


### Reload from file

In [4]:
events = pd.read_hdf(filename).fillna({'geo_location':'--'}) 
events.count()

display_id        23120126
document_id       23120126
timestamp         23120126
platform_x        23120126
geo_location      23120126
platform_y        23120126
traffic_source    23120126
dtype: int64

### Fix 'platform'

In [6]:
index = events['platform_x']<>events['platform_y']
err = events[index]
print len(err)
print err.platform_x.unique()

953
['\\N' '1' '2' '3']


In [7]:
index = ~events['platform_x'].str.isdigit()
print sum(index)

5


In [8]:
events.loc[index,'platform'] = events[index]['platform_y'].astype(int)
events.loc[~index,'platform'] = events[~index]['platform_x'].astype(int)

In [9]:
index = events['platform']<>events['platform_y'].astype(int)
print sum(index)

948


In [10]:
events.dtypes

display_id          int32
document_id         int32
timestamp           int32
platform_x         object
geo_location       object
platform_y         object
traffic_source    float64
platform          float64
dtype: object

In [11]:
events.drop(['platform_x', 'platform_y'], axis=1, inplace=True)
events.head()

,display_id,document_id,timestamp,geo_location,traffic_source,platform
7,8,1330329,638,US>PA>566,1.0,2.0
12,13,1778533,739,US>AZ>753,1.0,1.0
34,35,1271578,2633,US>NC>560,1.0,3.0
36,37,1779285,2687,US>WA>819,1.0,2.0
55,56,1108855,3925,CA>ON,1.0,2.0


### Fix 'geo_location'

In [13]:
locations = pd.DataFrame({'geo_location':events.geo_location.unique()}).sort_values(by='geo_location')
locations.head()

,geo_location
1118,--
1135,A1
656,A2
2296,AD
2697,AD>02


In [14]:
locations.tail()

,geo_location
552,ZW
2195,ZW>00
471,ZW>04
2141,ZW>06
2878,ZW>07


In [10]:
index = events['geo_location'] == '-->-->0'
print sum(index)

474


In [11]:
events.loc[index,'geo_location'] = '--'

In [16]:
events[index].head()

,display_id,document_id,timestamp,geo_location,traffic_source,platform
278633,278634,1753395,27385330,--,1.0,1.0
403091,403092,1789030,34012720,--,1.0,1.0
16981197,16981198,1763733,43700778,--,3.0,1.0
17025574,17025575,1776579,54766981,--,1.0,2.0
140347,156248,1803471,16005233,--,1.0,1.0


In [12]:
index = events['geo_location'] == '--'
print sum(index)

1275


### Encode geo_location

In [15]:
locations_list = events.geo_location.unique()
locations_list.sort()
locations_count = len(locations_list)

In [16]:
locations_list

array(['--', 'A1', 'A2', ..., 'ZW>04', 'ZW>06', 'ZW>07'], dtype=object)

In [17]:
locations = pd.DataFrame({'geo_location':locations_list, 'geo':range(-1, locations_count-1)}).sort_values(by='geo')
locations.count()

geo             2987
geo_location    2987
dtype: int64

In [18]:
locations.head()

,geo,geo_location
0,-1,--
1,0,A1
2,1,A2
3,2,AD
4,3,AD>02


In [44]:
locations.to_csv('../generated/geo_locations.csv', index=False)

In [43]:
events = events.merge(locations, on='geo_location')
events.head()

,display_id,document_id,timestamp,geo_location,traffic_source,platform,day,hour,geo
0,8,1330329,638,US>PA>566,1.0,2.0,1,7,2765
1,2657,1271490,185255,US>PA>566,1.0,2.0,1,7,2765
2,6344,461545,434576,US>PA>566,1.0,2.0,1,7,2765
3,7039,1793073,482930,US>PA>566,1.0,1.0,1,7,2765
4,11527,1518592,800258,US>PA>566,3.0,2.0,1,7,2765


In [47]:
events.drop('geo_location', axis=1, inplace=True)

### Expand timestamp: day of week, hour of day

In [18]:
import datetime

In [19]:
events['temp'] = events['timestamp'].apply(lambda timestamp: datetime.datetime.fromtimestamp((timestamp + 1465876799998)/1000 + 3600))

In [20]:
events.head()

,display_id,document_id,timestamp,geo_location,traffic_source,platform,temp
7,8,1330329,638,US>PA>566,1.0,2.0,2016-06-14 07:00:00
12,13,1778533,739,US>AZ>753,1.0,1.0,2016-06-14 07:00:00
34,35,1271578,2633,US>NC>560,1.0,3.0,2016-06-14 07:00:02
36,37,1779285,2687,US>WA>819,1.0,2.0,2016-06-14 07:00:02
55,56,1108855,3925,CA>ON,1.0,2.0,2016-06-14 07:00:03


In [21]:
events['day'] = events['temp'].apply(lambda temp: temp.weekday())
events['hour'] = events['temp'].apply(lambda temp: temp.hour)
events.head()

,display_id,document_id,timestamp,geo_location,traffic_source,platform,temp,day,hour
7,8,1330329,638,US>PA>566,1.0,2.0,2016-06-14 07:00:00,1,7
12,13,1778533,739,US>AZ>753,1.0,1.0,2016-06-14 07:00:00,1,7
34,35,1271578,2633,US>NC>560,1.0,3.0,2016-06-14 07:00:02,1,7
36,37,1779285,2687,US>WA>819,1.0,2.0,2016-06-14 07:00:02,1,7
55,56,1108855,3925,CA>ON,1.0,2.0,2016-06-14 07:00:03,1,7


In [22]:
events.drop('temp', axis=1, inplace=True)

### Save to HDF

In [48]:
events.count()

display_id        23120126
document_id       23120126
timestamp         23120126
traffic_source    23120126
platform          23120126
day               23120126
hour              23120126
geo               23120126
dtype: int64

In [49]:
events.to_hdf(filename+'.0', key='default', mode='w', complevel=5, complib='zlib')  

In [50]:
events.head()

,display_id,document_id,timestamp,traffic_source,platform,day,hour,geo
0,8,1330329,638,1.0,2.0,1,7,2765
1,2657,1271490,185255,1.0,2.0,1,7,2765
2,6344,461545,434576,1.0,2.0,1,7,2765
3,7039,1793073,482930,1.0,1.0,1,7,2765
4,11527,1518592,800258,3.0,2.0,1,7,2765


### Final save to CSV

In [4]:
events = pd.read_hdf(filename+'.0')
events.count()

display_id        23120126
document_id       23120126
timestamp         23120126
traffic_source    23120126
platform          23120126
day               23120126
hour              23120126
geo               23120126
dtype: int64

In [6]:
events.to_csv("../generated/final/events.csv", index=False)